In [389]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math

In [390]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [391]:
old_df=pd.read_csv('/content/drive/MyDrive/2018Floor1.csv')

In [392]:
old_df['Date'] = pd.to_datetime(old_df['Date'])

In [393]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

GPU is available


In [394]:
old_df

,Date,z1_Light(kW),z1_Plug(kW),z2_AC1(kW),z2_AC2(kW),z2_AC3(kW),z2_AC4(kW),z2_Light(kW),z2_Plug(kW),z3_Light(kW),z3_Plug(kW),z4_Light(kW)
0,2018-07-01 00:00:00,12.94,18.56,45.24,0.01,0.01,0.00,13.76,17.64,10.92,0.89,35.76
1,2018-07-01 00:01:00,12.97,18.55,45.28,0.02,0.01,0.01,13.76,17.21,10.95,0.87,35.81
2,2018-07-01 00:02:00,12.97,18.55,45.24,0.01,0.01,0.01,13.79,17.18,10.94,0.86,35.78
3,2018-07-01 00:03:00,12.98,18.58,45.26,0.02,0.01,0.00,13.81,16.64,10.94,0.85,35.83
4,2018-07-01 00:04:00,13.01,18.60,45.22,0.02,0.01,0.01,13.83,15.69,10.97,0.85,35.86
...,...,...,...,...,...,...,...,...,...,...,...,...
264955,2018-12-31 23:55:00,0.01,0.57,49.30,1.37,0.01,0.01,18.03,14.09,10.21,0.37,14.58
264956,2018-12-31 23:56:00,0.01,0.57,45.44,1.36,0.01,0.00,17.72,11.37,10.13,0.38,14.60
264957,2018-12-31 23:57:00,0.01,0.58,2.31,1.36,0.01,0.01,17.48,11.35,10.07,0.38,14.57
264958,2018-12-31 23:58:00,0.02,0.57,2.31,1.35,0.00,0.00,17.43,11.39,9.99,0.37,14.59


In [395]:
# Hourly data
df_numeric = old_df.drop(columns=['Date'])
df = df_numeric.groupby(df_numeric.index // 60).sum()
df_dates = old_df['Date'].groupby(old_df.index // 60).first()
df = df_dates.to_frame().join(df)

In [396]:
df

,Date,z1_Light(kW),z1_Plug(kW),z2_AC1(kW),z2_AC2(kW),z2_AC3(kW),z2_AC4(kW),z2_Light(kW),z2_Plug(kW),z3_Light(kW),z3_Plug(kW),z4_Light(kW)
0,2018-07-01 00:00:00,780.93,1141.65,2726.45,0.96,0.59,0.37,829.42,1001.16,657.39,45.81,2152.77
1,2018-07-01 01:00:00,784.25,1158.83,2749.04,0.96,0.59,0.38,834.54,1009.70,660.67,43.51,2158.75
2,2018-07-01 02:00:00,781.73,1155.64,2748.00,0.96,0.59,0.36,831.48,1001.03,659.43,43.41,2154.32
3,2018-07-01 03:00:00,772.92,1132.55,2735.13,0.94,0.58,0.35,820.81,960.78,653.92,43.13,2139.68
4,2018-07-01 04:00:00,771.75,1112.81,2728.31,0.95,0.58,0.33,825.23,869.53,653.90,43.62,2737.88
...,...,...,...,...,...,...,...,...,...,...,...,...
4411,2018-12-31 19:00:00,0.74,37.15,679.15,2133.49,0.31,1.49,1040.99,843.18,596.73,22.71,1481.00
4412,2018-12-31 20:00:00,0.73,42.22,635.07,2093.09,0.32,1.46,1029.52,842.59,597.30,22.52,1461.05
4413,2018-12-31 21:00:00,0.78,39.72,639.14,2082.07,0.29,1.43,1050.38,845.87,619.27,23.14,990.92
4414,2018-12-31 22:00:00,0.73,40.66,1261.81,656.22,0.47,0.70,1045.04,930.39,602.05,22.81,1473.36


In [397]:
df = df.fillna(0)

In [398]:
grouped_df = df.groupby('Date').sum()
grouped_df['total_power'] = grouped_df.sum(axis=1)
df = grouped_df[['total_power']].reset_index()

In [399]:
df

,Date,total_power
0,2018-07-01 00:00:00,9337.50
1,2018-07-01 01:00:00,9401.22
2,2018-07-01 02:00:00,9376.95
3,2018-07-01 03:00:00,9260.79
4,2018-07-01 04:00:00,9744.89
...,...,...
4411,2018-12-31 19:00:00,6836.94
4412,2018-12-31 20:00:00,6725.87
4413,2018-12-31 21:00:00,6293.01
4414,2018-12-31 22:00:00,6034.24


In [400]:
df['total_power'] = df['total_power'].astype(float)

In [401]:
# Daily data
df_temp = df.drop(columns=['Date'])
combined_df = df_temp.groupby(df_temp.index // 24).sum()
df_dates = df['Date'].groupby(df.index // 24).first()
combined_df = df_dates.to_frame().join(combined_df)

In [402]:
combined_df

,Date,total_power
0,2018-07-01,199051.98
1,2018-07-02,217731.45
2,2018-07-03,173226.22
3,2018-07-04,176526.90
4,2018-07-05,168039.47
...,...,...
179,2018-12-27,171440.35
180,2018-12-28,165403.76
181,2018-12-29,171626.60
182,2018-12-30,158321.50


In [403]:
# Custom dataset
class TimeSeriesDataset(Dataset):

  def __init__(self, training_sequence, target_sequence, sequence_length):
    self.training_sequence = np.asarray(training_sequence, dtype=np.float32)
    self.target_sequence = np.asarray(target_sequence, dtype=np.float32)
    self.sequence_length = sequence_length

  def __len__(self):
    return len(self.target_sequence) - self.sequence_length

  def __getitem__(self, idx):
    x = self.training_sequence[idx*24 : (idx+self.sequence_length)*24]
    y = self.target_sequence[idx+self.sequence_length]
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)



In [404]:
number_of_days = 49
sequence_length = 7
train_size = number_of_days
test_size = 20

In [405]:
# Normalizing df
data_max_df = df['total_power'].max()
data_min_df = df['total_power'].min()
df['total_power'] = (df['total_power'] - data_min_df) / (data_max_df - data_min_df)

In [406]:
# Normalizing combined_df
data_max_combined = combined_df['total_power'].max()
data_min_combined = combined_df['total_power'].min()
combined_df['total_power'] = (combined_df['total_power'] - data_min_combined) / (data_max_combined - data_min_combined)

In [407]:
training_sequence = df['total_power'][-((train_size + test_size)*24):].values
target_sequence = combined_df['total_power'][-train_size - test_size:].values

In [408]:
dataset = TimeSeriesDataset(training_sequence, target_sequence, sequence_length=sequence_length)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [409]:
for batch_idx, (x, y) in enumerate(dataloader):
  print(f"Batch {batch_idx+1}:")
  print(f"x: {x}")
  print(f"y: {y}")

Batch 1:
x: tensor([[0.3898, 0.3896, 0.4062,  ..., 0.5807, 0.4322, 0.3356],
        [0.4820, 0.4822, 0.5000,  ..., 0.2903, 0.3399, 0.3045],
        [0.4393, 0.4438, 0.4451,  ..., 0.5307, 0.5278, 0.4907],
        ...,
        [0.3778, 0.3255, 0.3328,  ..., 0.4877, 0.4806, 0.4374],
        [0.5281, 0.5258, 0.5154,  ..., 0.5437, 0.5438, 0.5051],
        [0.4524, 0.4546, 0.4505,  ..., 0.3038, 0.3294, 0.3171]])
y: tensor([0.3509, 0.4057, 0.5286, 0.5352, 0.4642, 0.7126, 0.5133, 0.5438, 0.4619,
        0.4579, 0.4054, 0.5913, 0.6911, 0.6911, 0.5837, 0.5561])
Batch 2:
x: tensor([[0.3283, 0.2953, 0.2957,  ..., 0.6253, 0.5806, 0.5070],
        [0.4544, 0.4132, 0.3584,  ..., 0.4875, 0.4789, 0.4375],
        [0.3463, 0.3250, 0.3066,  ..., 0.4122, 0.3888, 0.3540],
        ...,
        [0.4436, 0.4404, 0.4392,  ..., 0.4404, 0.4382, 0.3808],
        [0.4265, 0.4471, 0.4497,  ..., 0.5554, 0.5250, 0.4877],
        [0.4532, 0.4509, 0.4533,  ..., 0.4505, 0.4588, 0.3929]])
y: tensor([0.5792, 0.6929, 0.331

In [410]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [411]:
# Transformer model
class TimeSeriesTransformer(nn.Module):

  def __init__(self, input_size, d_model, n_heads, num_layers):
    super(TimeSeriesTransformer, self).__init__()
    self.embedding = nn.Linear(input_size, d_model)
    self.positional_embedding = nn.Parameter(torch.zeros(300, d_model))
    self.transformer = nn.Transformer(d_model, n_heads, num_layers)
    self.fc_out = nn.Linear(d_model, 1)

  def forward(self, src):
    src = src.unsqueeze(-1).to(device)
    src = self.embedding(src).to(device)
    src = src + self.positional_embedding[:src.size(1), :]
    src = src.permute(1, 0, 2).to(device)
    transformer_output = self.transformer(src, src).to(device)
    transformer_output = transformer_output.permute(1, 0, 2).to(device)
    output = self.fc_out(transformer_output).to(device)
    return output[:, -1, :]



In [412]:
input_size = 1
d_model = 64
n_heads = 8
num_layers = 2
model = TimeSeriesTransformer(input_size, d_model, n_heads, num_layers).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [413]:
# Training loop
epochs = 500
for epoch in range(epochs):
  epoch_loss = 0.0
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.unsqueeze(-1).to(device)
    output = model(x).to(device)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()

  epoch_loss /= len(dataloader)

  print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

Epoch 1, Loss: 0.1731
Epoch 2, Loss: 0.1190
Epoch 3, Loss: 0.0751
Epoch 4, Loss: 0.0420
Epoch 5, Loss: 0.0395
Epoch 6, Loss: 0.0325
Epoch 7, Loss: 0.0179
Epoch 8, Loss: 0.0236
Epoch 9, Loss: 0.0216
Epoch 10, Loss: 0.0246
Epoch 11, Loss: 0.0231
Epoch 12, Loss: 0.0325
Epoch 13, Loss: 0.0212
Epoch 14, Loss: 0.0213
Epoch 15, Loss: 0.0178
Epoch 16, Loss: 0.0223
Epoch 17, Loss: 0.0201
Epoch 18, Loss: 0.0208
Epoch 19, Loss: 0.0208
Epoch 20, Loss: 0.0193
Epoch 21, Loss: 0.0163
Epoch 22, Loss: 0.0174
Epoch 23, Loss: 0.0200
Epoch 24, Loss: 0.0157
Epoch 25, Loss: 0.0138
Epoch 26, Loss: 0.0152
Epoch 27, Loss: 0.0187
Epoch 28, Loss: 0.0164
Epoch 29, Loss: 0.0224
Epoch 30, Loss: 0.0163
Epoch 31, Loss: 0.0164
Epoch 32, Loss: 0.0167
Epoch 33, Loss: 0.0187
Epoch 34, Loss: 0.0178
Epoch 35, Loss: 0.0165
Epoch 36, Loss: 0.0201
Epoch 37, Loss: 0.0225
Epoch 38, Loss: 0.0161
Epoch 39, Loss: 0.0196
Epoch 40, Loss: 0.0196
Epoch 41, Loss: 0.0176
Epoch 42, Loss: 0.0185
Epoch 43, Loss: 0.0198
Epoch 44, Loss: 0.01

In [424]:
forecast_length = 14
true_values = []
predicted_values = []

In [425]:
model.eval()
with torch.no_grad():
  for num in range(1, forecast_length+1):
    last_sequence = torch.tensor(training_sequence[-((sequence_length)*24) - (num*24):-(num*24)], dtype=torch.float32).unsqueeze(0).to(device)
    normalized_forecast = model(last_sequence)
    forecast = normalized_forecast.item() * (data_max_combined - data_min_combined) + data_min_combined
    predicted_values.append(forecast)
    true_values.append(target_sequence[-(num+1)] * (data_max_combined - data_min_combined) + data_min_combined)

    print(f'Forecast {num}: {forecast:.4f}')
    print(f'Actual {num}: {target_sequence[-(num+1)] * (data_max_combined - data_min_combined) + data_min_combined}')

print(f'True values: {true_values}')
print(f'Predicted values: {predicted_values}')

Forecast 1: 114702.2904
Actual 1: 158321.5
Forecast 2: 155084.7605
Actual 2: 171626.6
Forecast 3: 158359.5944
Actual 3: 165403.76
Forecast 4: 163544.0369
Actual 4: 171440.35
Forecast 5: 165346.2428
Actual 5: 152918.64
Forecast 6: 164340.3592
Actual 6: 179353.71000000002
Forecast 7: 167817.2580
Actual 7: 141696.06
Forecast 8: 133700.2315
Actual 8: 135501.55
Forecast 9: 156681.0635
Actual 9: 124095.45
Forecast 10: 124883.8497
Actual 10: 182041.32
Forecast 11: 170407.2642
Actual 11: 158618.94
Forecast 12: 167568.4912
Actual 12: 150919.17000000004
Forecast 13: 159744.3271
Actual 13: 157132.56
Forecast 14: 161285.0043
Actual 14: 152906.96
True values: [158321.5, 171626.6, 165403.76, 171440.35, 152918.64, 179353.71000000002, 141696.06, 135501.55, 124095.45, 182041.32, 158618.94, 150919.17000000004, 157132.56, 152906.96]
Predicted values: [114702.29041454554, 155084.7605075425, 158359.5943663454, 163544.03692685426, 165346.2428269571, 164340.35923466444, 167817.25796267748, 133700.23148694247

In [426]:
true_values = np.array(true_values)
predicted_values = np.array(predicted_values)

In [427]:
# Calculate MAPE
def calculate_mape(true, pred):
    return np.mean(np.abs((true - pred) / true)) * 100

In [428]:
# Calculate RMSE
def calculate_rmse(true, pred):
    return np.sqrt(np.mean((true - pred) ** 2))

In [429]:
# Calculate R-squared
def calculate_r2(true, pred):
    ss_tot = np.sum((true - np.mean(true)) ** 2)
    ss_res = np.sum((true - pred) ** 2)
    return 1 - (ss_res / ss_tot)

In [430]:
# Calculate MAPE for the provided values
mape = calculate_mape(np.array(true_values), np.array(predicted_values))

print(f'MAPE: {mape:.2f}%')

MAPE: 11.83%


In [431]:
# Calculate RMSE for the provided values
rmse = calculate_rmse(true_values, predicted_values)

print(f'RMSE: {rmse:.2f}')

RMSE: 24165.54


In [432]:
# Calculate R-squared for the provided values
r2 = calculate_r2(true_values, predicted_values)

print(f'R-squared: {r2:.2f}')

R-squared: -1.35
